In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import copy
import logging
import contextily as cx
import gtfs_kit as gk
import fastsim as fsim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import lightning.pytorch as pl
import rasterio as rio
import seaborn as sns
from rasterio.plot import show
import seaborn as sns
import shapely
import statsmodels.api as sm
from torch.utils.data import DataLoader

from openbustools.traveltime.models import rnn
from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory, busnetwork
from openbustools.drivecycle.physics import conditions, energy, vehicle

In [ ]:
point_sep_m = 300

### Draw Energy Predictions for Full KCM Network

In [ ]:
data_dir = Path("..", "data")
epsg_code = 32148
timezone = "America/Los_Angeles"
coord_ref_center = [386910,69022]

In [ ]:
importlib.reload(busnetwork)
# Get most recent static feed
static_dir = data_dir / f"kcm_static"
latest_static_file = standardfeeds.latest_available_static("2024_02_06", static_dir)
latest_static_file = static_dir / latest_static_file
# Get reprojected DEM file
dem_file = Path("..","data","kcm_spatial","usgs10m_dem_32148.tif")
# Load most recently trained model
model_dir = Path("..", "logs", "kcm", "GRU-4")
latest_version = str(sorted([int(x.split('_')[1]) for x in os.listdir(model_dir)])[-1])
latest_ckpt = sorted(os.listdir(model_dir / f"version_{latest_version}" / "checkpoints"))[-1]
model = rnn.GRU.load_from_checkpoint(model_dir / f"version_{latest_version}" / "checkpoints" / latest_ckpt, strict=False).eval()
# Get trajectories and update travel times
trajectories = busnetwork.get_trajectories(latest_static_file, epsg=epsg_code, coord_ref_center=coord_ref_center, dem_file=dem_file)
busnetwork.update_travel_times(trajectories, model)
cycles = [busnetwork.get_trajectory_energy(traj) for traj in trajectories]

In [ ]:
efficiencies = np.array([1/cycle.electric_kwh_per_mi for cycle in cycles])
fig, ax = plt.subplots()
sns.histplot(efficiencies[efficiencies<5], kde=True, ax=ax)
ax.set_title(f"BEB Shape Efficiency Distribution\n{len(efficiencies)} Routes")
ax.set_xlabel("Efficiency (mi/kWh)")

In [ ]:
outlier_indices = np.where(efficiencies > 4)[0]
outlier_trajs = [trajectories[i] for i in outlier_indices]
outlier_cycles = [cycles[i] for i in outlier_indices]
outlier_avg_grades = [np.mean(cycle.cyc.grade) for cycle in outlier_cycles]
print(outlier_avg_grades)
plotting.drive_cycle_energy_plot(outlier_cycles)

### Draw Energy Predictions for All Networks

In [ ]:
cleaned_sources = pd.read_csv(Path('..', 'data', 'cleaned_sources.csv'))
data_dir = Path("..", "data", "other_feeds")

all_network_trajs = []
all_network_results = []
for i, row in cleaned_sources.iterrows():
    # Get most recent static feed
    static_dir = data_dir / f"{row['uuid']}_static"
    latest_static_file = standardfeeds.latest_available_static("2024_02_06", static_dir)
    latest_static_file = static_dir / latest_static_file
    # Get reprojected DEM file
    dem_dir = data_dir / f"{row['uuid']}_spatial"
    dem_file = list(dem_dir.glob(f"*{row['epsg_code']}.tif"))[0]
    # Load most recently trained model
    model_dir = data_dir / f"{row['uuid']}_models" / "lightning_logs"
    latest_version = str(sorted([int(x.split('_')[1]) for x in os.listdir(model_dir)])[-1])
    latest_ckpt = sorted(os.listdir(model_dir / f"version_{latest_version}" / "checkpoints"))[-1]
    model = rnn.GRU.load_from_checkpoint(model_dir / f"version_{latest_version}" / "checkpoints" / latest_ckpt, strict=False).eval()
    # Get trajectories and update travel times
    trajectories = busnetwork.get_trajectories(latest_static_file, epsg=row['epsg_code'], coord_ref_center=[row['coord_ref_x'], row['coord_ref_y']], dem_file=dem_file)
    busnetwork.update_travel_times(trajectories, model)
    cycles = [busnetwork.get_trajectory_energy(traj) for traj in trajectories]
    all_network_trajs.append(trajectories)
    all_network_results.append(cycles)

filehandler = open("../results/energy/all_network_trajs.pkl", "wb")
pickle.dump(all_network_trajs,filehandler)
filehandler.close()

filehandler = open("../results/energy/all_network_results.pkl", "wb")
pickle.dump(all_network_results,filehandler)
filehandler.close()

In [ ]:
file = open("../results/energy/all_network_trajs.pkl", "rb")
all_network_trajs = pickle.load(file)
file.close()

file = open("../results/energy/all_network_results.pkl", "rb")
all_network_results = pickle.load(file)
file.close()

In [ ]:
fig, ax = plt.subplots()
for network_cycles in all_network_results:
    efficiencies = np.array([1/cycle.electric_kwh_per_mi for cycle in network_cycles])
    sns.histplot(efficiencies[efficiencies<5], kde=True, ax=ax, label=row['uuid'])
ax.set_title(f"")
ax.set_xlabel("Efficiency (mi/kWh)")

In [ ]:
# Use results to answer research questions.